In [ ]:
# Прошу прощения, пока все еще не сделано. Прошу еще раз отправить на доработку. 

In [ ]:
Модели ARIMA и GARCH, прогнозирование значений на их основе
Преподаватель: Алексей Кузьмин

Выберите ряд из первого задания и постройте его модель ARIMA/GARCH. Отчет нужно будет оформить в jupyter-ноутбук.
Дополнительное задание: попробуйте спрогнозировать график классическим ML.
